In [34]:
import osmnx as ox
import pandas as pd
import openpyxl
import folium

In [38]:
def creating_graph(G,data, type_of_movement=None):
#     G_length=cprecision=2)
#     points = ox.utils_geo.sample_points(data["longitude"],data["latitude"]
    G_nodes,G_length=ox.distance.nearest_nodes(G,data["longitude"],data["latitude"],return_dist=True)
    
    if  type_of_movement=='electric_scooter':
        G = ox.add_edge_speeds(G,hwy_speeds={"road":15})#add max_speed scooter
    elif type_of_movement=='electric_bus':
        G = ox.add_edge_speeds(G,hwy_speeds={"road":20})#add max_speed for bus
    else :
        G = ox.add_edge_speeds(G,hwy_speeds={"road":7})#add max_speed for walk,none and except
        
    G=ox.speed.add_edge_travel_times(G)# add time to edges
    # add length for edges
        
    if len(G_nodes)<=3:
        return creating_roads_less_4_points(G,G_nodes)
    
    elif len(G_nodes)>=4:
            return creating_roads_more_3_points(G,G_nodes)

In [3]:
def creating_roads_more_3_points(G,G_nodes):
    orig=list(G_nodes[:-1])
    dest=list(G_nodes[1:])
    
    #  CRETING path by  TRAVEL_TIME
    route_by_time=ox.distance.shortest_path(G,orig=orig,dest=dest, weight="travel_time")
    finally_route_by_time=[a for b in route_by_time for a in b[:-1] ]
    finally_route_by_time.append(G_nodes[-1])
#     for x in range(len(route_by_time)):

#         if x !=len(route_by_time)-1:
#             finally_route_by_time.append(route_by_time[x][:-1])
            
    #  CRETING path by  TRAVEL_LENGTH  
    route_by_length=ox.distance.shortest_path(G,orig=orig,dest=dest, weight="length") 
    finally_route_by_length=[a for b in route_by_length for a in b[:-1]]
    finally_route_by_length.append(G_nodes[-1])
#     for x in range(len(route_by_length)):
#         if x !=len(route_by_length)-1:
#             finally_route_by_length.append(route_by_length[x][:-1])

    return finally_route_by_time, finally_route_by_length

In [4]:
def creating_roads_less_4_points(G,G_nodes):
    if G_nodes==2:
        orig=G_nodes[0]
        dest=G_nodes[1]
        paths=ox.distance.k_shortest_paths(G,orig=orig,dest=dest, k=2,weight="travel_time")   
        return paths
    
    
    elif G_nodes==3:
        orig=G_nodes[0]
        dest=G_nodes[1]
        paths_first_past=ox.distance.k_shortest_paths(G,orig=orig,dest=dest, k=2,weight="travel_time")
        
        for paths_first_past in paths_first_past:
            orig=G_nodes[1]
            dest=G_nodes[2]
            paths_second_past=ox.distance.k_shortest_paths(G,orig=orig,dest=dest, k=2,weight="travel_time")
        
        count=1
        for path_first_past,path_second_past in zip(paths_first_past,paths_second_past):
            if count==1:
                first_route=path_first_past+path_second_past[1:]
                count+=1
            elif count==2:
                second_route= path_first_past+path_second_past[1:]
            
            
        return first_route,second_route

In [6]:
data=pd.read_excel(r"C:\Users\Guest\Desktop\data_with_coor.xlsx")
types_movement=["walking","electric_scooter","electric_bus"]# whats can be
type_movement="electric_scooter"
graph=ox.graph_from_bbox(55.8465,55.8219,37.6404,37.5875,network_type="walk")
data.shape

C:\Users\Guest\anaconda3\lib\site-packages\osmnx\utils_geo.py:280: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for polygon in geometry:
C:\Users\Guest\anaconda3\lib\site-packages\osmnx\utils_geo.py:374: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for poly in multipoly:
C:\Users\Guest\anaconda3\lib\site-packages\osmnx\utils_geo.py:374: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for poly in multipoly:


(446, 36)

In [26]:
data['distence_between_first_point'] = ox.distance.euclidean_dist_vec(data["longitude"].loc[0],data["latitude"].loc[0],data["longitude"].loc[1:],data["latitude"].loc[1:])
data.loc[0,"distence_between_first_point"]=0
data.sort_values(by='distence_between_first_point',inplace=True)

In [39]:
routes=creating_graph(graph,data,type_movement)
route_by_time=routes[0]
route_time= int(sum(ox.utils_graph.get_route_edge_attributes(graph, route_by_time,  attribute="travel_time",minimize_key='length')))
route_by_length=routes[1]
route_length = int(sum(ox.utils_graph.get_route_edge_attributes(graph, route_by_length,  attribute="length")))

print(f'values routes: {routes}')
print(f'time for first route: {route_time}')
print(f'length for second route: {route_length}')

AttributeError: module 'osmnx.distance' has no attribute 'nearest_nodes'

In [21]:
m = folium.Map(location=[ 55.833308, 37.626951], zoom_start=15, tiles="OpenStreetMap")
tooltip = "Click me!"

ox.folium.plot_route_folium(graph,route=route_by_time,route_map=m, color="red")
# ox.folium.plot_route_folium(graph,route=route_by_time,route_map=m, color="blue")

In [8]:
import geopandas as gpd
import psycopg2
from pyproj import CRS
from shapely.geometry import Point
data=pd.read_excel(r"C:\Users\User\Desktop\data_with_coor.xlsx")

geometry = [Point(xy) for xy in zip(data["longitude"],data["latitude"])]
gdf = gpd.GeoDataFrame(crs = CRS('EPSG:4326'), geometry=geometry)




In [22]:
count=0
for _, r in gdf.iterrows():
    lat = r['geometry'].y
    lon = r['geometry'].x
    
    if count==0:# first marker
        folium.Marker(location=[lat, lon],
                    popup=f'lat:{lat} \n lon:{lon}',
                    icon=folium.Icon(color="red"),
                    tooltip=tooltip
                 ).add_to(m)
    elif count ==  len(gdf)-1:# last marker
        folium.Marker(location=[lat, lon],
                    popup=f'lat:{lat} \n lon:{lon}',
                    icon=folium.Icon(color="green"),
                    tooltip=tooltip
                 ).add_to(m)
        
    else:
        folium.Marker(location=[lat, lon],
                    popup=f'lat:{lat} \n lon:{lon}',
                     icon=folium.Icon(color='orange',icon='1'),
                    tooltip=tooltip
                 ).add_to(m)
    count+=1

In [23]:
m

In [20]:
data['distence_between_first_point'] = ox.distance.euclidean_dist_vec(data["longitude"].loc[0],data["latitude"].loc[0],data["longitude"].loc[1:],data["latitude"].loc[1:])
data.loc[0,"distence_between_first_point"]=0
data.head()


,Unnamed: 0.1,Unnamed: 0,type,id,order,type.1,_icon,_order,_url,cat,...,type_s1,type_en,type_cn,url,visibility,tickets_link,zoom,longitude,latitude,distence_between_first_point
0,0,0,Feature,242,250,Point,pavilion,250,/places/pavilon-66-sovetskaya-kultura-byvshiy-...,place,...,Павильон,Pavilion,亭,/places/pavilon-66-sovetskaya-kultura-byvshiy-...,visible,NaN,16.0,37.631906,55.831638,0.000000
1,1,1,Feature,243,100,Point,pavilion,100,/places/pavilon-71-atomnaya-energiya-rsfsr/,place,...,Павильон,Pavilion,亭,/places/pavilon-71-atomnaya-energiya-rsfsr/,visible,NaN,16.0,37.633838,55.829978,0.002547
2,2,2,Feature,244,1000,Point,entry,1000,/places/vezd-cherez-kpp-sever-3/,place,...,Въезд,Entry,进入,/places/vezd-cherez-kpp-sever-3/,visible,NaN,11.8,37.642502,55.828157,0.011153
3,3,3,Feature,245,1000,Point,entry,1000,/places/vezd-cherez-kpp-sever-1/,place,...,Въезд,Entry,进入,/places/vezd-cherez-kpp-sever-1/,visible,NaN,11.8,37.640560,55.827124,0.009760
4,4,4,Feature,246,350,Point,restaurant,350,/places/kafe-anderson-na-dache/,place,...,Еда,Food,食物,/places/kafe-anderson-na-dache/,visible,NaN,13.3,37.627645,55.828055,0.005567
